Ensure that this notebook is moved one level up to the directory '2_analyze_proportions_of_total_attention_S3.1_3.2'.
The data found in this directory is generated from the notebooks in the directory '1_generate_raw_data'.

The output of this notebook is a .csv file 'proportions_data.csv'.
This is a table where each row contains various measures and comparisons of the proportion of total attention each model paid to subsets of tokens representing legal concepts. Each row is a distinct observation of a specific attention head (given by layer-head index) and legal concept.

In [5]:
import numpy as np
import pandas as pd
import os
import glob

# Mapping dictionary for easier references
mapping_dict = {
    'EquallSaul-7B-Base': 'saul_base',
    'EquallSaul-7B-Instruct-v1': 'saul_ift',
    'mistralaiMistral-7B-v0.1': 'mistral_base'
}

# Glob pattern to match the required files
file_pattern = '*_*_proportions.npy'

# Get the list of all matching files
files = glob.glob(file_pattern)

# Create an empty list to store the extracted rows
rows = []

# Function to extract experiment_id from filename
def get_experiment_id(filepath):
    return os.path.basename(filepath).split('_')[0]  # facet-1, facet-2, etc.

# Function to extract unmapped_model_id from filename
def get_unmapped_model_id(filepath):
    return os.path.basename(filepath).split('_')[1].replace('_proportions.npy', '')

# Function to map model names using mapping_dict
def get_model_name(unmapped_model_id):
    for key, short_name in mapping_dict.items():
        if key in unmapped_model_id:
            return short_name
    return None

# Create a dictionary to hold loaded numpy arrays
loaded_data = {}

# Load each file into memory
for file in files:
    experiment_id = get_experiment_id(file)
    unmapped_model_id = get_unmapped_model_id(file)
    model_name = get_model_name(unmapped_model_id)

    # Skip files that don't match any known model
    if model_name is None:
        continue

    if experiment_id not in loaded_data:
        loaded_data[experiment_id] = {}

    # Load the numpy array and store it
    loaded_data[experiment_id][model_name] = np.load(file)

# Iterate over the loaded data and construct rows
for experiment_id, models_data in loaded_data.items():
    for layer in range(32):
        for head in range(32):
            prop_mistral = models_data.get('mistral_base', np.zeros((32, 32)))[layer][head]
            prop_saulbase = models_data.get('saul_base', np.zeros((32, 32)))[layer][head]
            prop_saulift = models_data.get('saul_ift', np.zeros((32, 32)))[layer][head]

            row = {
                'experiment_id': experiment_id,
                'layer_depth': layer,
                'head_idx': head,
                'prop_mistralbase': prop_mistral,
                'prop_saulbase': prop_saulbase,
                'prop_saulift': prop_saulift,
                'saulbase-mistral': prop_saulbase - prop_mistral,
                'saulift-mistral': prop_saulift - prop_mistral,
                'saulift-saulbase': prop_saulift - prop_saulbase
            }
            rows.append(row)

# Convert the list of rows to a DataFrame
df = pd.DataFrame(rows)

df.to_csv('/content/proportions_data.csv', index=False)